In [24]:
import numpy as np
import scipy.optimize as optimize

In [63]:
node_capacities = [10,5]
transport_capacities = [1,1]
demands = [[2,4,5,3],[0,1,1,1]]
movements_orig = np.array(
                  [[[0,0],
                   [0,0]],
                  [[0,0],
                   [0,0]],
                  [[0,0],
                   [0,0]],
                  [[0,0],
                   [0,0]],
                 ])

movements = movements_orig.ravel()
prob_death_transport = 0.05
num_days = len(demands[0])
num_nodes = len(node_capacities)

In [64]:
def calculate_outgoing(array,day,node):
    return sum(array[day][node])

def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing

def demand_day_node_raw(demand_day, node_capacity, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day + min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    incoming_patients = calculate_incoming(movements_res,day,node)
    outgoing_patients = calculate_outgoing(movements_res,day,node)
    if(day==0):
        prev_demand = 0
    else:
        prev_demand = demand_day_node(movements,day-1,node)
        
    demand = max(0, demand_day_node_raw(demands[node][day], node_capacities[node],
                                 prev_demand,                                 
                                 outgoing_patients, incoming_patients))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
            
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list


def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list


def f_cons(a, b, c):
    return lambda x: transport_capacities[a] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),b ,c)


cons = ({'type': 'ineq', 'fun': f_cons(0,0,0)},
        {'type': 'ineq', 'fun': f_cons(1,0,1)},
        {'type': 'ineq', 'fun': f_cons(0,1,0)},
        {'type': 'ineq', 'fun': f_cons(1,1,1)},
        {'type': 'ineq', 'fun': f_cons(0,2,0)},
        {'type': 'ineq', 'fun': f_cons(1,2,1)},
        {'type': 'ineq', 'fun': f_cons(0,3,0)},
        {'type': 'ineq', 'fun': f_cons(1,3,1)})

In [57]:
initial_guess = movements.copy()

In [58]:
bounds_movement = generate_bounds()
result = optimize.minimize(calc_total_deaths, initial_guess, method='SLSQP', constraints=cons, bounds=bounds_movement)

In [59]:
result.x

array([6.82045155e-16, 6.66668349e-01, 5.24241027e-16, 5.20888327e-16,
       5.22594782e-16, 6.66668349e-01, 6.02232050e-16, 6.39033715e-16,
       5.64879656e-17, 6.66668349e-01, 1.86470824e-17, 3.08065993e-17,
       1.14465308e-16, 1.69482205e-16, 1.36097590e-16, 1.28119378e-16])

In [60]:
np.rint(result.x)

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [61]:
calc_total_deaths(np.rint(result.x))

2.3

In [62]:
calc_total_deaths(movements)

4.0